## Importing libraries and dataset
So let’s get started. First of all, we will import all the required libraries.

In [ ]:
import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter("ignore")

# Preprocessing
from collections import Counter
from nltk.probability import FreqDist
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords, wordnet  
from sklearn.feature_extraction.text import CountVectorizer   
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer 
from sklearn.model_selection import train_test_split, RandomizedSearchCV

# Building classification models
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

# Model evaluation
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix, f1_score, precision_score, recall_score

#### Import datasets

In [38]:
train_data = pd.read_csv(r'C:\Users\HP\Downloads\language-identification\train_set.csv')
test_data = pd.read_csv(r'C:\Users\HP\Downloads\language-identification\test_set.csv')

In [39]:
train_data.head()

,lang_id,text
0,xho,umgaqo-siseko wenza amalungiselelo kumaziko ax...
1,xho,i-dha iya kuba nobulumko bokubeka umsebenzi na...
2,eng,the province of kwazulu-natal department of tr...
3,nso,o netefatša gore o ba file dilo ka moka tše le...
4,ven,khomishini ya ndinganyiso ya mbeu yo ewa maana...


#### counting the value count for each language

In [40]:
train_data["lang_id"].value_counts()

sot    3000
nbl    3000
xho    3000
tsn    3000
zul    3000
afr    3000
nso    3000
ven    3000
ssw    3000
tso    3000
eng    3000
Name: lang_id, dtype: int64

#### Separating Independent and Dependent features

In [ ]:
X = train_data["text"]
y = train_data["lang_id"]

#### Text Preprocessing

In [ ]:
def clean_text(df):
  item = 0
  for txt in df['text']:
    txt = txt .lower()
    txt = re.sub(r'[^\w\s]', '', txt)
    txt = re.sub(r'\d+', '', txt)
    txt = re.sub(r'\s\s+', ' ', txt)
    txt = txt.lstrip(' ')  
    txt = txt.lstrip()
    txt = txt.rstrip()
    txt = txt.replace('  ', ' ')
    df.loc[item, 'text'] = txt
    item += 1

In [ ]:
clean_text(train_data)

In [ ]:
clean_text(test_data)

#### Train/Test Splitting

In [ ]:
# Seperate features and tagret variables
X = train_data['text']
y = train_data['lang_id']

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

#### Model Training

In [ ]:
clfs = [RandomForestClassifier(max_depth=5, n_estimators=100), KNeighborsClassifier(n_neighbors=5, 
                                             metric='minkowski', 
                                             p=2),
              MultinomialNB(), LinearSVC(class_weight='balanced')]

In [48]:
def trainer(clfs, x_train, x_test, y_train, y_test):
    score_results = {}
    # Pipeline to balance the classses and then to build the model
    for clf in clfs:
        piper = Pipeline([('tfidf', TfidfVectorizer(ngram_range=(2, 3))),
                              ('clf', clf)])
        
        piper.fit(x_train, y_train)
        predictions = piper.predict(x_test)

        # Output for each model
        score_results[clf.__class__.__name__] = {
            'F1-Macro': metrics.f1_score(y_test,
                                         predictions,
                                         average='macro'),
            'F1-Accuracy': metrics.f1_score(y_test, predictions,
                                            average='micro'),
            'F1-Weighted': metrics.f1_score(y_test,
                                            predictions,
                                            average='weighted')}

    return pd.DataFrame.from_dict(score_results, orient='index')

#### Model performance

In [49]:
clfs_df = trainer(clfs, x_train, x_test, y_train, y_test)
cfl_df = clfs_df.sort_values('F1-Macro', ascending=False)
cfl_df

,F1-Macro,F1-Accuracy,F1-Weighted
LinearSVC,0.965187,0.964697,0.965368
MultinomialNB,0.957631,0.958333,0.958474
KNeighborsClassifier,0.915776,0.913485,0.914608
RandomForestClassifier,0.567611,0.570303,0.568972


#### Predict the output and submission

In [50]:
mnb = Pipeline([('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])
classifier = mnb.fit(x_train, y_train)
y_pred = classifier.predict(test_data['text'])

submission=pd.DataFrame(data={'index':test_data['index'],'lang_id':y_pred})
submission.to_csv('Language_Identification_Hackathon',index=False)

In [51]:
submission.head()

,index,lang_id
0,1,tsn
1,2,nbl
2,3,ven
3,4,ssw
4,5,afr


In [1]:
import pickle

model_save_path = "natural_language_hackathon.pkl"
with open(model_save_path,'wb') as file:
    pickle.dump(classifier, file)

NameError: name 'classifier' is not defined